In [ ]:
import os
import sys
sys.path.append("../")

import panel as pn
import altair as alt
from altair import datum
import pandas as pd
# from vega_datasets import data
import datetime as dt

from src.utils.snowflake import read_forecast_meta, read_forecasts
from src.plot.altair import plot_all, plot_decompose

In [ ]:
alt.renderers.enable("default")
pn.extension('tabulator')
# pn.extension("vega")

In [ ]:
df_forecast_meta = read_forecast_meta()
df_table = (
    df_forecast_meta.rename(
        columns={
            "vermogen_nominaal": "P_nom",
            "relative_abs_max": "P_max",
            "date_abs_max": "t_max",
        }
    )
    .drop(columns=["processed_on", "l", "stationsnummer", "P_nom"])
    .sort_values(by=["P_max", "t_max"], ascending=[False, True])
)

In [ ]:
tabulator_formatters = {
    "float": {"type": "progress", "max": 1},
    "bool": {"type": "tickCross"},
}

table = pn.widgets.Tabulator(
    df_table,
    formatters=tabulator_formatters,
    show_index=False,
    disabled=True,
    selectable=True,
    height=500,
    width=280,
)

In [ ]:
boxid = "VRY.HOESW-1"

In [ ]:
df_total = read_forecasts(boxid=boxid)

In [ ]:
plot_total = plot_all(
    df_data=df_total, df_meta=df_forecast_meta.query(f"boxid == '{boxid}'")
).properties(width=550, height=400)

plot_min = plot_decompose(df_total.query("extreme == 'min'"))

plot_max = plot_decompose(df_total.query("extreme == 'max'"))

In [ ]:
tabs = pn.Tabs()
tabs.append(("Total", plot_total))
tabs.append(("Min", plot_min))
tabs.append(("Max", plot_max))

In [ ]:
pn.Row(table, tabs, min_height=500)

In [ ]:
alt.renderers.enable("default")
pn.extension("vega")

In [ ]:
df_meta = read_meta()

In [ ]:
df_meta

In [ ]:
select_set = ["regio_netautomatisering", "vestiging"]
df_select = df_meta.copy()[select_set].drop_duplicates(subset=select_set).sort_values(select_set)

In [ ]:

regio_checkbuttons = pn.widgets.CheckButtonGroup(
    name='regio', 
    value=list(df_select["regio_netautomatisering"].unique()), 
    options=list(df_select["regio_netautomatisering"].unique())
)

vestiging_checkbuttons = pn.widgets.CheckButtonGroup(
    name='vestiging', 
    value=vestigingen, options=vestigingen
)

pn.Column(regio_checkbuttons, vestiging_checkbuttons)

In [ ]:
df_select["regio_netautomatisering"].unique()

In [ ]:
df_meta